# Réponse aux questions

Dans ce notebook, nous allons nous intéresser à la réponse aux questions, plus particulièrement à la réponse aux questions *extractives*. Les étapes suivantes seront suivies :

1. Préparation des données.
2. Finetuning du modèle.
3. Utilisation du modèle.

J'ai suivi une partie du cours [**NLP Course**](https://huggingface.co/learn/nlp-course/fr/chapter7/7) d'Hugging Face pour réaliser ce notebook.

## Préparation des données

Dans ce notebook, on utilise le jeu de données [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/).

### Récupération du jeu de données

On télécharge et on met en cache le jeu de données en une seule étape grâce à `load_dataset()` :

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Voici à quoi ressemble le jeu de données :

In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

On y voit trois champs qui nous intéresse : `context`, `question` et `answers`. L'idée est qu'on pose une question et la tâche consiste à répondre à cette question à partir du contexte. Voici à quoi peuvent ressembler ces trois champs :

In [5]:
print("Context: ", raw_datasets["train"][0]["context"], "\n")
print("Question: ", raw_datasets["train"][0]["question"], "\n")
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. 

Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? 

Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


Les champs `context` et `question` sont simples à comprendre. Or, le champ `answers` est plus difficile à comprendre. Il s'agit d'un dictionnaire avec deux champs : le champ `text` qui correspond à une liste de réponses (textes) et le champ `answer_start` qui correspond à une liste d'indices qui est l’indice du caractère de départ de chaque réponse dans le contexte.

Pour l'entraînement, il n’y a qu’une seule réponse possible :

In [6]:
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

Pour l’évaluation, cependant, il existe plusieurs réponses possibles pour chaque échantillon, qui peuvent être identiques ou différentes :

In [7]:
print(raw_datasets["validation"][0]["answers"])
print(raw_datasets["validation"][2]["answers"])

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [177, 177, 177]}
{'text': ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."], 'answer_start': [403, 355, 355]}


Mais pourquoi ? Ça provient du fait que certaines des questions ont plusieurs réponses possibles ! Le script d’évaluation (tout sera enveloppé pour nous par une métrique de *Datasets*) va comparer une réponse prédite à toutes les réponses acceptables et prendre le meilleur score. Pour mieux comprendre cette idée, voici un exemple :

In [10]:
print("Context: ", raw_datasets["validation"][2]["context"], "\n")
print("Question: ", raw_datasets["validation"][2]["question"], "\n")
print("Answers: ", raw_datasets["validation"][2]["answers"], "\n")

Context:  Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. 

Question:  Where did Super Bowl 50 take place? 

Answers:  {'text': ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."], 'answer_start': [403,

On voit bien que ces différentes réponses sont possibles.